## Generating Student Vectors (Embedding + Aggregation)

this notebook generate students vector in "course" with CodeBook k (k:the number of centroids, 100 is default)
".\data\vectors\norm_Student_Vctors_course{course}_{k}dim.csv"

In [78]:
import pandas as pd
import numpy as np
import fasttext as ft
import re
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

select courses and k (the number of centroids). k correspond to the dimensions of student vectors. 

In [79]:
# course_list = [A-2021,A-2022,D-2021,D-2022]
course= "A-2022"
k=100

In [80]:
Code_Book_dir = r".\data\code_book"
CodeBook_file = Code_Book_dir + r"\CodeBook_k{}.csv".format(k)

### prepareing for Generating Student vectors in "course"

In [81]:
# actions :  All actions in one course 
# user_df :  userid and actions of this student

actions = r".\data2\actions_txt\actions_{}_perStudents.txt".format(course)
users_actions = []
user_action = ""
user=[]
with open(actions, "r") as f:
    actions_list = f.readlines()
    for action in actions_list:
        if action.startswith("****"):
            user.append(action.rstrip("\n").replace("*",""))
            users_actions.append(user_action)
            user_action = ""
        else:
            user_action += action + " "
user_df = pd.DataFrame(users_actions)
user_df["userid"] = user
user_df.set_index("userid",inplace=True)

In [82]:
user_df

,0
userid,
A-2022_U1,OsJJsNsNNsNNsPm_ NNNNNsNsEm NsC\n OsNm Nm Nm N...
A-2022_U2,OsNsNNsNsNsNNs_ NsNNsNNNsNsNsNs_ NsNNNsNsPmNsN...
A-2022_U3,OsJsNmNsJNsNsNm_ NsNNsNsPsNNsNs_ NsNsNmNNmNm N...
A-2022_U4,OsJsJsJJsJsJm Cm ONNNNNNsNNsNsNs_ NsJsJsJsNsNN...
A-2022_U5,OsNNNNNNNNNNNN_ NNNNNNNNNsNNNN_ NmC\n OsNl\n N...
A-2022_U6,ONsNsNPsPsNsNs_ NsNmNsNsNmNmNs_ NsNsNsNsNsNsNs...
A-2022_U7,OsNJsPPsPsPmNN_ NsNsNNsNsCl\n ONNNNNNNNNNNNN_ ...
A-2022_U8,OsCm OmNsNsPsPmNm PsNsNmNsPsNNsNs_ PsNsNNsPmNm...
A-2022_U9,OsENNEmNsNNNNN_ NNNNNNNNNNsNPP_ PmPsPPPPsNsNs ...


### Aggregation

In [83]:
## read CodeBook

action_centroids = pd.read_csv(CodeBook_file,index_col=0)
## load fastText model (we must use the same model to Making CodeBook)
model = ft.load_model(r".\model\fastText_trainALL-2020_100dim_30epoch.bin")

### Aggregation step
# users_vecs: output students vector 
users_vecs =[]
# users actions
users_data = user_df.values
# aggregation step for one student
for actions_one_user in users_data:
        # user_vec = one student aggregated vector 
        user_vec = [0*i for i in range(len(action_centroids))]
        # split each actions 
        actions_oneuser = actions_one_user[0].split("\n") 
        for one_action in actions_oneuser:
            # if empty action, skip vectorization
            if one_action == "" or one_action == " ":
                    continue
            # vector of action generated by fastText
            # embedding
            one_action_vec = model.get_sentence_vector(one_action)
            # calculate cosine similarity between action vec and centroids
            sim_mat = cosine_similarity(one_action_vec.reshape(1,-1), action_centroids.values)
            # get the most similar centroids ID
            max_index = np.argmax(sim_mat)
            # count the most similar centroids 
            user_vec[max_index] += 1
        users_vecs.append(user_vec)
boa_df = pd.DataFrame(users_vecs,index=user_df.index)

#normarization
norm_boa_df = pd.DataFrame(normalize(boa_df,norm="l2",axis=1),index=boa_df.index)
norm_boa_df.to_csv(r".\data2\vectors\norm_Student_Vctors_course{}_{}dim.csv".format(course,k))

In [84]:
norm_boa_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
userid,,,,,,,,,,,,,,,,,,,,,
A-2022_U1,0.281663,0.000000,0.000000,0.000000,0.093888,0.031296,0.093888,0.000000,0.000000,0.031296,...,0.000000,0.031296,0.000000,0.031296,0.031296,0.000000,0.000000,0.000000,0.031296,0.000000
A-2022_U2,0.048550,0.000000,0.000000,0.000000,0.169925,0.048550,0.097100,0.000000,0.024275,0.242750,...,0.000000,0.097100,0.000000,0.000000,0.000000,0.000000,0.024275,0.024275,0.097100,0.000000
A-2022_U3,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.125000,0.125000,0.000000,0.187500,...,0.000000,0.125000,0.000000,0.062500,0.000000,0.062500,0.000000,0.062500,0.062500,0.000000
A-2022_U4,0.021315,0.000000,0.021315,0.000000,0.042630,0.042630,0.063946,0.000000,0.042630,0.191837,...,0.063946,0.042630,0.021315,0.063946,0.000000,0.042630,0.021315,0.021315,0.021315,0.021315
A-2022_U5,0.000000,0.000000,0.000000,0.000000,0.000000,0.039467,0.276268,0.000000,0.000000,0.355202,...,0.000000,0.000000,0.000000,0.039467,0.039467,0.039467,0.000000,0.039467,0.000000,0.039467
A-2022_U6,0.275033,0.000000,0.082510,0.000000,0.082510,0.055007,0.137516,0.000000,0.000000,0.055007,...,0.027503,0.137516,0.000000,0.082510,0.000000,0.027503,0.000000,0.000000,0.055007,0.000000
A-2022_U7,0.446986,0.000000,0.000000,0.031928,0.063855,0.095783,0.191565,0.031928,0.000000,0.031928,...,0.000000,0.095783,0.000000,0.031928,0.031928,0.031928,0.000000,0.063855,0.000000,0.000000
A-2022_U8,0.000000,0.000000,0.052705,0.000000,0.316228,0.000000,0.316228,0.052705,0.000000,0.158114,...,0.052705,0.105409,0.000000,0.000000,0.052705,0.000000,0.000000,0.000000,0.000000,0.000000
A-2022_U9,0.239236,0.000000,0.000000,0.000000,0.159490,0.039873,0.119618,0.000000,0.000000,0.079745,...,0.000000,0.039873,0.000000,0.000000,0.039873,0.039873,0.000000,0.000000,0.000000,0.000000


## To comparsion, fasttext trained by A-2020, and D-2020. k = 100.

### A-2020

In [85]:
## read CodeBook
if k == 100:
        CodeBook_file = Code_Book_dir + r"\CodeBook_k{}_A20.csv".format(k)
        action_centroids = pd.read_csv(CodeBook_file,index_col=0)
        ## load fastText model (we must use the same model to Making CodeBook)
        model = ft.load_model(r".\model\fastText_trainA-2020_100dim_30epoch.bin")

        ### Aggregation step
        # users_vecs: output students vector 
        users_vecs =[]
        # users actions
        users_data = user_df.values
        # aggregation step for one student
        for actions_one_user in users_data:
                # user_vec = one student aggregated vector 
                user_vec = [0*i for i in range(len(action_centroids))]
                # split each actions 
                actions_oneuser = actions_one_user[0].split("\n") 
                for one_action in actions_oneuser:
                # if empty action, skip vectorization
                        if one_action == "" or one_action == " ":
                                continue
                        # vector of action generated by fastText
                        # embedding
                        one_action_vec = model.get_sentence_vector(one_action)
                        # calculate cosine similarity between action vec and centroids
                        sim_mat = cosine_similarity(one_action_vec.reshape(1,-1), action_centroids.values)
                        # get the most similar centroids ID
                        max_index = np.argmax(sim_mat)
                        # count the most similar centroids 
                        user_vec[max_index] += 1
                users_vecs.append(user_vec)
        boa_df = pd.DataFrame(users_vecs,index=user_df.index)

        #normarization
        norm_boa_df = pd.DataFrame(normalize(boa_df,norm="l2",axis=1),index=boa_df.index)
        norm_boa_df.to_csv(r".\data2\vectors\norm_Student_Vctors_course{}_{}dim_A20.csv".format(course,k))

In [86]:
norm_boa_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
userid,,,,,,,,,,,,,,,,,,,,,
A-2022_U1,0.031296,0.000000,0.000000,0.031296,0.000000,0.000000,0.000000,0.000000,0.062592,0.000000,...,0.000000,0.250367,0.062592,0.000000,0.031296,0.000000,0.000000,0.031296,0.031296,0.000000
A-2022_U2,0.083109,0.000000,0.055406,0.000000,0.027703,0.055406,0.055406,0.000000,0.055406,0.027703,...,0.000000,0.193921,0.110812,0.027703,0.000000,0.138515,0.000000,0.166218,0.000000,0.055406
A-2022_U3,0.181902,0.000000,0.000000,0.060634,0.000000,0.000000,0.121268,0.121268,0.000000,0.000000,...,0.000000,0.242536,0.060634,0.121268,0.060634,0.060634,0.000000,0.000000,0.060634,0.000000
A-2022_U4,0.000000,0.021306,0.042611,0.000000,0.021306,0.127833,0.213056,0.000000,0.021306,0.021306,...,0.000000,0.490028,0.085222,0.021306,0.021306,0.106528,0.000000,0.042611,0.085222,0.021306
A-2022_U5,0.000000,0.000000,0.043769,0.000000,0.000000,0.043769,0.350150,0.000000,0.000000,0.043769,...,0.000000,0.131306,0.175075,0.000000,0.175075,0.000000,0.000000,0.043769,0.087538,0.043769
A-2022_U6,0.025786,0.000000,0.000000,0.051571,0.025786,0.000000,0.051571,0.000000,0.025786,0.025786,...,0.025786,0.077357,0.128928,0.000000,0.051571,0.025786,0.025786,0.000000,0.077357,0.025786
A-2022_U7,0.060606,0.060606,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.090909,0.000000,...,0.000000,0.151515,0.090909,0.030303,0.030303,0.000000,0.000000,0.030303,0.090909,0.000000
A-2022_U8,0.055048,0.000000,0.055048,0.000000,0.000000,0.000000,0.110096,0.000000,0.165145,0.000000,...,0.055048,0.055048,0.000000,0.000000,0.055048,0.000000,0.055048,0.165145,0.055048,0.055048
A-2022_U9,0.116510,0.000000,0.000000,0.000000,0.000000,0.000000,0.038837,0.038837,0.077674,0.038837,...,0.000000,0.116510,0.038837,0.038837,0.000000,0.038837,0.000000,0.000000,0.077674,0.000000


### D-2020

In [87]:
## read CodeBook
if k==100:
        CodeBook_file = Code_Book_dir + r"\CodeBook_k{}_D20.csv".format(k)
        action_centroids = pd.read_csv(CodeBook_file,index_col=0)
        ## load fastText model (we must use the same model to Making CodeBook)
        model = ft.load_model(r".\model\fastText_trainD-2020_100dim_30epoch.bin")

        ### Aggregation step
        # users_vecs: output students vector 
        users_vecs =[]
        # users actions
        users_data = user_df.values
        # aggregation step for one student
        for actions_one_user in users_data:
                # user_vec = one student aggregated vector 
                user_vec = [0*i for i in range(len(action_centroids))]
                # split each actions 
                actions_oneuser = actions_one_user[0].split("\n") 
                for one_action in actions_oneuser:
                # if empty action, skip vectorization
                        if one_action == "" or one_action == " ":
                                continue
                        # vector of action generated by fastText
                        # embedding
                        one_action_vec = model.get_sentence_vector(one_action)
                        # calculate cosine similarity between action vec and centroids
                        sim_mat = cosine_similarity(one_action_vec.reshape(1,-1), action_centroids.values)
                        # get the most similar centroids ID
                        max_index = np.argmax(sim_mat)
                        # count the most similar centroids 
                        user_vec[max_index] += 1
                users_vecs.append(user_vec)
        boa_df = pd.DataFrame(users_vecs,index=user_df.index)

        #normarization
        norm_boa_df = pd.DataFrame(normalize(boa_df,norm="l2",axis=1),index=boa_df.index)
        norm_boa_df.to_csv(r".\data2\vectors\norm_Student_Vctors_course{}_{}dim_D20.csv".format(course,k))

In [88]:
norm_boa_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
userid,,,,,,,,,,,,,,,,,,,,,
A-2022_U1,0.032844,0.000000,0.000000,0.000000,0.032844,0.000000,0.098533,0.000000,0.032844,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.131377,0.032844,0.000000,0.000000,0.000000,0.000000
A-2022_U2,0.281129,0.000000,0.000000,0.000000,0.000000,0.000000,0.127786,0.000000,0.000000,0.025557,...,0.127786,0.051114,0.000000,0.000000,0.178900,0.025557,0.102229,0.000000,0.025557,0.000000
A-2022_U3,0.149256,0.000000,0.000000,0.000000,0.000000,0.000000,0.447767,0.000000,0.049752,0.049752,...,0.099504,0.000000,0.000000,0.000000,0.049752,0.049752,0.000000,0.000000,0.000000,0.000000
A-2022_U4,0.102047,0.017008,0.017008,0.000000,0.000000,0.000000,0.187087,0.017008,0.000000,0.153071,...,0.391181,0.000000,0.000000,0.034016,0.051024,0.000000,0.102047,0.000000,0.000000,0.000000
A-2022_U5,0.258023,0.000000,0.000000,0.000000,0.000000,0.000000,0.221163,0.000000,0.000000,0.184302,...,0.479186,0.000000,0.000000,0.000000,0.294884,0.036860,0.036860,0.000000,0.000000,0.000000
A-2022_U6,0.147531,0.000000,0.000000,0.000000,0.000000,0.000000,0.196708,0.024589,0.024589,0.049177,...,0.000000,0.000000,0.000000,0.049177,0.122943,0.024589,0.000000,0.024589,0.000000,0.000000
A-2022_U7,0.152075,0.030415,0.000000,0.000000,0.000000,0.000000,0.243320,0.030415,0.030415,0.030415,...,0.091245,0.000000,0.000000,0.000000,0.091245,0.000000,0.000000,0.000000,0.000000,0.000000
A-2022_U8,0.000000,0.048795,0.000000,0.000000,0.000000,0.000000,0.390360,0.000000,0.048795,0.195180,...,0.097590,0.000000,0.000000,0.000000,0.146385,0.000000,0.000000,0.000000,0.000000,0.000000
A-2022_U9,0.069213,0.000000,0.000000,0.000000,0.000000,0.000000,0.207639,0.000000,0.034606,0.034606,...,0.034606,0.000000,0.000000,0.034606,0.034606,0.000000,0.138426,0.000000,0.000000,0.000000
